# Neural Networks Assignment 1 Exercise 2

In [8]:
#Write a Python program to train the model and use the following multilayer ANN to predict the output for class Die_Live:
#importing libraries
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [3]:
#loading the dataset
df_hepatitis_niyanta = pd.read_csv("C:/Users/niyan/OneDrive/Desktop/hepatitis.data")

In [4]:
df_hepatitis_niyanta.columns=['CLASS', 'AGE', 'SEX', 'STEROID','ANTIVRALS','FATIGUE','MALAISE','ANOREXIA','LIVER BIG','LIVER FIRM','SPLEEN PALPABLE','SPIDERS','ASCITES','VARICES','BILIRUBIN','ALK PHOSPHATE','SGOT','ALBUMIN','PROTIME','HISTOLOGY']

In [5]:
#printing the first 5 rows using head function
print(df_hepatitis_niyanta.head())

   CLASS  AGE  SEX STEROID  ANTIVRALS FATIGUE MALAISE ANOREXIA LIVER BIG  \
0      2   50    1       1          2       1       2        2         1   
1      2   78    1       2          2       1       2        2         2   
2      2   31    1       ?          1       2       2        2         2   
3      2   34    1       2          2       2       2        2         2   
4      2   34    1       2          2       2       2        2         2   

  LIVER FIRM SPLEEN PALPABLE SPIDERS ASCITES VARICES BILIRUBIN ALK PHOSPHATE  \
0          2               2       2       2       2      0.90           135   
1          2               2       2       2       2      0.70            96   
2          2               2       2       2       2      0.70            46   
3          2               2       2       2       2      1.00             ?   
4          2               2       2       2       2      0.90            95   

  SGOT ALBUMIN PROTIME  HISTOLOGY  
0   42     3.5       ?    

In [31]:
#count missing values in the column
df_hepatitis[df_hepatitis_niyanta.columns].isnull().sum()

CLASS              0
AGE                0
SEX                0
STEROID            0
ANTIVRALS          0
FATIGUE            0
MALAISE            0
ANOREXIA           0
LIVER BIG          0
LIVER FIRM         0
SPLEEN PALPABLE    0
SPIDERS            0
ASCITES            0
VARICES            0
BILIRUBIN          0
ALK PHOSPHATE      0
SGOT               0
ALBUMIN            0
PROTIME            0
HISTOLOGY          0
dtype: int64

In [6]:
#counting the unique values for column CLASS
df_hepatitis_niyanta.CLASS.value_counts()

2    122
1     32
Name: CLASS, dtype: int64

In [7]:
def make_int(i):
    try:
        return int(i)
    except:
        return np.nan

for i in ['STEROID','ANTIVRALS','FATIGUE','MALAISE','ANOREXIA','LIVER BIG','LIVER FIRM','SPLEEN PALPABLE','SPIDERS','ASCITES','VARICES','ALK PHOSPHATE','SGOT','PROTIME','HISTOLOGY']:
    df_hepatitis_niyanta[i] = df_hepatitis_niyanta[i].map(make_int)


def make_float(i):
    try:
        return float(i)
    except:
        return np.nan
    
for i in ['BILIRUBIN','ALBUMIN']:
    df_hepatitis_niyanta[i] = df_hepatitis_niyanta[i].map(make_float)

for i in ['STEROID','ANTIVRALS','FATIGUE','MALAISE','ANOREXIA','LIVER BIG','LIVER FIRM','SPLEEN PALPABLE','SPIDERS','ASCITES','VARICES','ALK PHOSPHATE','SGOT','PROTIME','HISTOLOGY','BILIRUBIN','ALBUMIN']:
    df_hepatitis_niyanta[i].fillna((df_hepatitis_niyanta[i].mean()), inplace=True)
    print(df_hepatitis_niyanta)

     CLASS  AGE  SEX   STEROID  ANTIVRALS  FATIGUE  MALAISE  ANOREXIA  \
0        2   50    1  1.000000          2      1.0      2.0       2.0   
1        2   78    1  2.000000          2      1.0      2.0       2.0   
2        2   31    1  1.509804          1      2.0      2.0       2.0   
3        2   34    1  2.000000          2      2.0      2.0       2.0   
4        2   34    1  2.000000          2      2.0      2.0       2.0   
..     ...  ...  ...       ...        ...      ...      ...       ...   
149      1   46    1  2.000000          2      1.0      1.0       1.0   
150      2   44    1  2.000000          2      1.0      2.0       2.0   
151      2   61    1  1.000000          2      1.0      1.0       2.0   
152      2   53    2  1.000000          2      1.0      2.0       2.0   
153      1   43    1  2.000000          2      1.0      2.0       2.0   

     LIVER BIG  LIVER FIRM  SPLEEN PALPABLE  SPIDERS  ASCITES  VARICES  \
0          1.0         2.0              2.0      

In [34]:
#sigmoid function
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

#derivative of sigmoid function
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

In [13]:
class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""

        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {} : {} / {}".format(j,self.evaluate(test_data),n_test));
            else:
                print("Epoch {} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        output_test = [(self.feedforward(x), y)
                for (x, y) in test_data]
        return sum(((x - y)**2).sum() for (x, y) in output_test)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

In [14]:
#assigning the features and target column
features = df_hepatitis.drop('CLASS', axis=1)
target = pd.DataFrame({"Die_Live1": df_hepatitis['CLASS'] - 1,"Die_Live2": np.where(df_hepatitis['CLASS'] == 2, 0, 1)})

In [15]:
#training,testing and splitting the dataset
X_train, X_test, Y_train, Y_test = train_test_split(features,target,test_size=0.20, random_state=10)

In [16]:
inputs_train = [np.reshape(x, (19, 1)) for x in np.array(X_train)]
output_train = [np.reshape(y, (2, 1)) for y in np.array(Y_train)]

inputs_test = [np.reshape(x, (19, 1)) for x in np.array(X_test)]
output_test = [np.reshape(y, (2, 1)) for y in np.array(Y_test)]

data_train = zip(inputs_train, output_train)
test_data = zip(inputs_test, output_test)

In [17]:
network = Network([19, 30, 15, 2])
network.SGD(data_train, 100, 100, 3, test_data = test_data)

C:\Users\niyan\AppData\Local\Temp\ipykernel_14552\4119304062.py:3: RuntimeWarning: overflow encountered in exp
  return 1.0/(1.0+np.exp(-z))


Epoch 0 : 10.023289454082859 / 31
Epoch 1 : 9.864534396680735 / 31
Epoch 2 : 9.80873883710346 / 31
Epoch 3 : 9.650668101226742 / 31
Epoch 4 : 11.064207201958085 / 31
Epoch 5 : 9.781130263921808 / 31
Epoch 6 : 9.945813423770643 / 31
Epoch 7 : 9.927830851259456 / 31
Epoch 8 : 9.837322150733344 / 31
Epoch 9 : 9.934987450386615 / 31
Epoch 10 : 9.845429159462313 / 31
Epoch 11 : 9.72649951899263 / 31
Epoch 12 : 9.824300701511692 / 31
Epoch 13 : 9.793658887715242 / 31
Epoch 14 : 10.001343899583446 / 31
Epoch 15 : 10.225850673890118 / 31
Epoch 16 : 10.025922071599773 / 31
Epoch 17 : 10.243230600874364 / 31
Epoch 18 : 15.02420137859565 / 31
Epoch 19 : 10.422131093243674 / 31
Epoch 20 : 10.227026601457059 / 31
Epoch 21 : 11.619006041015151 / 31
Epoch 22 : 9.896277417652696 / 31
Epoch 23 : 13.865288844252838 / 31
Epoch 24 : 10.277854442215123 / 31
Epoch 25 : 10.2709264348814 / 31
Epoch 26 : 12.989423558064997 / 31
Epoch 27 : 10.216301580225304 / 31
Epoch 28 : 10.158077828555959 / 31
Epoch 29 : 10